# MNIST with a Convolutional Neural Network

In [1]:
# Larger CNN for the MNIST Dataset
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


## The next line tells Keras that the images will be presented as Tensors with the channels first.

Huh? Channels are the red-green-blue channels of the image. So this tells us automatically that CNNs are typically used in image processing neural networks. There are two natural ways of creating Tensors for images. One is to put the channels first (e.g. 3 x 28 x 28). The other is to put the channels last (28 x 28 x 3). TensorFlow does it one way; Theano does it the other. Keras just needs to know which way the raw data is stored so that it can reshape the input Tensors to whichever backend it is using.

Note: Our images are greyscale. So the Tensor is really 1 x 28 x 28 (one grey channel from 0 to 255). Also, note that the Tensor is really 4D. The first dimension is the number of images in our dataset. So: # images in data x 1 x 28 x 28. 

In [23]:
K.set_image_data_format('channels_first')

In [24]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [ ]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

## We normalize the input between 0 and 1

So not zero mean, but still normalized.

In [25]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

## One-hot encoding

We just want to "one-hot" encode the output labels. So instead of 0 through 9, we have labels [1,0,0,0,0,0,0,0,0,0] through [0,0,0,0,0,0,0,0,0,1]. So label 5 would be [0,0,0,0,1,0,0,0,0,0].

In [26]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

## This is our actual CNN

We first have a 30 neuron CNN layer with a ReLu activation and a 5x5 convolutional filter. Then we do a max pooling on a 2 x 2 window. Then we have a 15 neuron layer with a ReLu activation and a 3 x 3 convolutional filter. Then we do a max pooling of this on a 2x2 window. We also add a 20% dropout to the layer. Then we flatten the image to a single vector (now that we have the spatial convolutional filters). We run that into a 128-neuron layer (ReLu) and a 50-neuron layer (ReLu). The output is the 10-class softmax.

In [27]:
# define the larger model
def larger_model():
# create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
# build the model
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 74s - loss: 0.3990 - acc: 0.8767 - val_loss: 0.0994 - val_acc: 0.9690
Epoch 2/10
60000/60000 [==============================] - 74s - loss: 0.0971 - acc: 0.9702 - val_loss: 0.0563 - val_acc: 0.9824
Epoch 3/10
60000/60000 [==============================] - 74s - loss: 0.0695 - acc: 0.9784 - val_loss: 0.0379 - val_acc: 0.9880
Epoch 4/10
60000/60000 [==============================] - 75s - loss: 0.0567 - acc: 0.9818 - val_loss: 0.0315 - val_acc: 0.9894
Epoch 5/10
60000/60000 [==============================] - 74s - loss: 0.0477 - acc: 0.9852 - val_loss: 0.0294 - val_acc: 0.9900
Epoch 6/10
60000/60000 [==============================] - 76s - loss: 0.0435 - acc: 0.9865 - val_loss: 0.0280 - val_acc: 0.9906
Epoch 7/10
60000/60000 [==============================] - 76s - loss: 0.0386 - acc: 0.9878 - val_loss: 0.0282 - val_acc: 0.9910
Epoch 8/10
60000/60000 [==============================

In [29]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

Large CNN Error: 0.70%


# So we've gone from about a 2% error to a less than 1% error
